In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from snowflake.sqlalchemy import URL
pd.set_option('display.max_columns', None)

In [ ]:
from sqlalchemy import create_engine


class extractorFromQuery:
    def __init__(self):
        self.credentials = {
            "account": "qfi-niro",
            "user": "keyur",
            "password": "Keyur@17061996",
            "warehouse": "NIRO_ANALYTICS_WAREHOUSE",
        }

    def queryExecutor(self, query: str, *args):
        engine = create_engine(URL(**self.credentials))
        res = pd.read_sql_query(query, engine, params=args)
        return res

In [ ]:
Q = """
        select * from (select
        distinct id.crn,
        id.los_app_id,
        id.gender,
        id.age,
        id.pincode,
        o.phone_number as "Phone_Number",
        p.pan_no as "Pan_Number",
        o.category,
        o.riskband2,
        o.max_amount, 
        o.interest_rate_perc,
        o.propensity,
        o.naps_score,
        o.dec_reason,
        la.loan_application_status,
        CASE when la.loan_disbursed_date_by_ops is null then date(la.disbursement_date)
            else date(la.loan_disbursed_date_by_ops)
        end as disbursement_date,
        rank() over(partition by o.user_id order by o.created_at desc) as rnk
        from 
        ANALYTICS.CRIF_FILES.idfc_demog as id 
        left join analytics.offers.offer_details o on id.crn = o.hashed_phone
        left join niro_rds.niro_data.core__public__loan_application la on la.user_id = o.user_id
        left join analytics.offers.personal_info as p on o.pii_id=p.pii_id
        left join analytics.offers.tradeline_info tli on o.tli_id = tli.tli_id
        where o.dec_reason = 'NOT DECLINED' 
        and o.platform_partner = 'Snapdeal' 
        and (p.pan_no is not null and pan_no<>'nan') 
        and o.created_at between '2023-01-01' and '2023-10-27'
        )
where rnk = 1;
"""

idfc_offers = (
    extractorFromQuery()
    .queryExecutor(Q)
)

idfc_offers.shape

#### Creating Aggregate CRIF tradeline data

In [ ]:
Q = """select los_app_id, count(los_app_id) as no_tr_l6m from(select los_app_id
from analytics.crif_files.account
where disbursed_dt >= '2023-04-27'
)
group by los_app_id"""

no_tr_l6m = (
    extractorFromQuery()
    .queryExecutor(Q)
)
no_tr_l6m.shape

In [ ]:
Q = """select los_app_id, max(credit_card_util) as max_cc_util from(select los_app_id, 
CASE when (credit_limit > 0) then (current_bal/credit_limit) 
when (disbursed_amt > 0) then (current_bal/disbursed_amt)
else null end as credit_card_util
from analytics.crif_files.account
where acct_type in ('Secured Credit Card', 'Credit Card', 'Kisan Credit Card', 'Loan on Credit Card', 'Loan against Card', 'Corporate Credit Card', 'Fleet Card')
)
group by los_app_id"""

max_cc_util = (
    extractorFromQuery()
    .queryExecutor(Q)
)
max_cc_util.shape

In [ ]:
max_cc_util.head(2)

In [ ]:
Q = """select distinct crn, score as crif_score
from analytics.crif_files.idfc_output
where dec_reason = 'NOT DECLINED'"""

crif_score = (
    extractorFromQuery()
    .queryExecutor(Q)
)
crif_score.shape

In [ ]:
crif_score.loc[crif_score.crif_score.astype(float) < 650, 'crif_score'].value_counts(dropna=False).sum()

In [ ]:
Q = """select distinct crn,
1 as carded,
sector
from analytics.crif_files.idfc_output
where sector in ('Secured Credit Card', 'Credit Card', 'Kisan Credit Card', 'Loan on Credit Card', 'Loan against Card', 'Corporate Credit Card', 'Fleet Card')"""

carded = (
    extractorFromQuery()
    .queryExecutor(Q)
)
carded.head()

In [ ]:
Q = """select crn,
max(ever_delq),
max(max_delq24),
max(max_delq12),
max(max_delq9),
max(max_delq6),
max(max_delq3),
min(credit_limit_santionedamount),
max(writeoff_status)
from analytics.crif_files.idfc_output
group by crn"""

perf = (
    extractorFromQuery()
    .queryExecutor(Q)
)
perf.head()

#### Creating Aggregate CRIF Enquiry data

In [ ]:
Q = """select crn, los_app_id, enquirydate
from analytics.crif_files.idfc_inq"""

idfc_inq = (
    extractorFromQuery()
    .queryExecutor(Q)
)
idfc_inq.shape

In [ ]:
idfc_inq.head()

In [ ]:
idfc_inq.isna().sum()

In [ ]:
idfc_inq_df = idfc_inq.groupby(['crn'])['los_app_id'].count().reset_index().rename(columns = {'los_app_id':'enquiry_count_all'}).merge(
idfc_inq[idfc_inq.enquirydate.notna()].groupby(['crn'])['los_app_id'].count().reset_index().rename(columns = {'los_app_id':'enquiry_count_date_present'}), on = 'crn', how = 'left').merge(
idfc_inq[pd.to_datetime(idfc_inq['enquirydate']) >= pd.to_datetime('2023-05-01')].groupby(['crn'])['los_app_id'].count().reset_index().rename(columns = {'los_app_id':'enquiry_count_l6m'}), on = 'crn', how = 'left').merge(
idfc_inq[pd.to_datetime(idfc_inq['enquirydate']) >= pd.to_datetime('2023-08-01')].groupby(['crn'])['los_app_id'].count().reset_index().rename(columns = {'los_app_id':'enquiry_count_l3m'}), on = 'crn', how = 'left')
idfc_inq_df.head()


#### Approved base

In [ ]:
Q = """select source_request_id as crn,
bureau_score,
exist_customer_flag,
loan_amount,
risk_segment,
roi,
yob,
1 as approved
from analytics.temp_test.idfc_whitelisted"""

idfc_approved = (
    extractorFromQuery()
    .queryExecutor(Q)
)
idfc_approved.shape

In [ ]:
idfc_approved.head()

In [ ]:
idfc_base = pd.merge(idfc_offers, idfc_inq_df, on='crn', how='left') \
    .merge(idfc_approved, on='crn', how='left') \
    .merge(crif_score, on='crn', how='left') \
    .merge(perf, on='crn', how='left') \
    .merge(carded, on='crn', how='left') \
    .merge(no_tr_l6m, on='los_app_id', how='left') \
    .merge(max_cc_util, on='los_app_id', how='left')
idfc_base.shape

Blocked pincodes

In [ ]:
pincode_blocked = pd.read_excel('Pincode List_Blocking 09102023 IDFC.xlsx')
pincode_blocked.head()

In [ ]:
idfc_base.loc[idfc_base.pincode.isin(pincode_blocked.Pincodes), 'loan_application_status'].value_counts()

In [ ]:
idfc_base.loc[idfc_base.pincode.isin(pincode_blocked.Pincodes), 'approved'].sum()

In [ ]:
idfc_base[['max_cc_util', 'no_tr_l6m']] = idfc_base[['max_cc_util', 'no_tr_l6m']].fillna(0)

Fixing CRIF issue

In [ ]:
idfc_base['crif_score'] = idfc_base.crif_score.astype(float)
idfc_base = idfc_base[idfc_base.crif_score >= 650].reset_index(drop=True)

In [ ]:
idfc_base.shape

In [ ]:
idfc_base.drop_duplicates(subset = 'crn', inplace=True)
idfc_base.shape

## Analysis

### Figuring out hard rules

Carded

In [ ]:
idfc_base.loc[idfc_base.approved.notna(), 'sector'].value_counts(dropna=False)

In [ ]:
idfc_base.carded.value_counts(dropna=False)

CRIF score

In [ ]:
idfc_base.bureau_score.min(), idfc_base.bureau_score.max()

In [ ]:
plt.figure(figsize=(8, 6))
idfc_base.hist(column = 'bureau_score', bins=20)
plt.title(f'Histogram for bureau_score')
plt.xlabel('bureau_score')
plt.ylabel('Frequency')
plt.show()

In [ ]:
idfc_base.crif_score.min(), idfc_base.crif_score.max()

In [ ]:
plt.figure(figsize=(8, 6))
idfc_base.hist(column = 'crif_score', bins=20)
plt.title(f'Histogram for crif_score')
plt.xlabel('crif_score')
plt.ylabel('Frequency')
plt.show()

Year of birth

In [ ]:
idfc_base.yob.min(), idfc_base.yob.max()
# Max age - 60, min age - 20

In [ ]:
plt.figure(figsize=(8, 6))
idfc_base.hist(column = 'yob', bins=20)
plt.title(f'Histogram for yob')
plt.xlabel('yob')
plt.ylabel('Frequency')
plt.show()

Enquiry

In [ ]:
idfc_base['enquiry_count_all'].value_counts(dropna=False)

In [ ]:
idfc_base.loc[idfc_base.approved.notna(), 'enquiry_count_all'].value_counts(dropna=False)

In [ ]:
idfc_base['enquiry_count_date_present'].value_counts(dropna=False)

In [ ]:
idfc_base.loc[idfc_base.approved.notna(), 'enquiry_count_date_present'].value_counts(dropna=False)

In [ ]:
idfc_base['enquiry_count_l6m'].value_counts(dropna=False)

In [ ]:
idfc_base.loc[idfc_base.approved.notna(), 'enquiry_count_l6m'].value_counts(dropna=False)

In [ ]:
idfc_base['enquiry_count_l3m'].value_counts(dropna=False)

In [ ]:
idfc_base.loc[idfc_base.approved.notna(), 'enquiry_count_l3m'].value_counts(dropna=False)

max_cc_util

In [ ]:
idfc_base['max_cc_util'] = np.where((idfc_base['max_cc_util'] < 0) | (idfc_base['max_cc_util'] >= 1) | idfc_base['max_cc_util'].isna(), 0, idfc_base['max_cc_util'])

In [ ]:
idfc_base['max_cc_util'].min(), idfc_base['max_cc_util'].max()

In [ ]:
idfc_base.loc[idfc_base.approved.notna(), 'max_cc_util'].min(), idfc_base.loc[idfc_base.approved.notna(), 'max_cc_util'].max()

In [ ]:
import seaborn as sns
sns.kdeplot(idfc_base.loc[idfc_base.approved.notna()], x = 'max_cc_util')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
idfc_base.hist(column = 'max_cc_util', bins=200)
plt.title(f'Histogram for max_cc_util')
plt.xlabel('max_cc_util')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
idfc_base.loc[idfc_base.approved.notna()].hist(column = 'max_cc_util', bins=200)
plt.title(f'Histogram for max_cc_util')
plt.xlabel('max_cc_util')
plt.ylabel('Frequency')
plt.show()

In [ ]:
idfc_base.head(2)

MAX(EVER_DELQ)

In [ ]:
idfc_base['MAX(EVER_DELQ)'].min(), idfc_base['MAX(EVER_DELQ)'].max()

In [ ]:
idfc_base.loc[idfc_base.approved.notna(), 'MAX(EVER_DELQ)'].min(), idfc_base.loc[idfc_base.approved.notna(), 'MAX(EVER_DELQ)'].max()

In [ ]:
plt.figure(figsize=(8, 6))
idfc_base.hist(column = 'MAX(EVER_DELQ)', bins=20)
plt.title(f'Histogram for MAX(EVER_DELQ)')
plt.xlabel('MAX(EVER_DELQ)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
idfc_base.loc[idfc_base.approved.notna()].hist(column = 'MAX(EVER_DELQ)', bins=20)
plt.title(f'Histogram for MAX(EVER_DELQ)')
plt.xlabel('MAX(EVER_DELQ)')
plt.ylabel('Frequency')
plt.show()

MAX(MAX_DELQ3)

In [ ]:
idfc_base['MAX(MAX_DELQ3)'].min(), idfc_base['MAX(MAX_DELQ3)'].max()

In [ ]:
idfc_base.loc[idfc_base.approved.notna(), 'MAX(MAX_DELQ3)'].min(), idfc_base.loc[idfc_base.approved.notna(), 'MAX(MAX_DELQ3)'].max()

In [ ]:
plt.figure(figsize=(8, 6))
idfc_base.hist(column = 'MAX(MAX_DELQ3)', bins=10)
plt.title(f'Histogram for MAX(MAX_DELQ3)')
plt.xlabel('MAX(MAX_DELQ3)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
idfc_base.loc[idfc_base.approved.notna()].hist(column = 'MAX(MAX_DELQ3)', bins=10)
plt.title(f'Histogram for MAX(MAX_DELQ3)')
plt.xlabel('MAX(MAX_DELQ3)')
plt.ylabel('Frequency')
plt.show()

### Approved Analysis

#### Propensity

In [ ]:
idfc_base.propensity.value_counts(dropna=False)

In [ ]:
idfc_base[idfc_base.approved.notna()].propensity.value_counts(dropna=False)

In [ ]:
so_wc_screen = pd.read_excel('Welcome screen base.xlsx')
so_wc_screen.head()

In [ ]:
idfc_base.loc[idfc_base['Phone_Number'].isin(so_wc_screen['Phone number'].astype(str)), 'propensity'].value_counts(dropna=False, normalize=True)

#### Category

In [ ]:
idfc_base.category.value_counts(dropna=False)

In [ ]:
idfc_base[idfc_base.approved.notna()].category.value_counts(dropna=False)

#### NAPS

In [ ]:
def naps_score_bkt(row, var):
    if row[var] <= 675:
        return '<=675'
    elif row[var] <= 690:
        return '675-690'
    elif row[var] <= 715:
        return '690-715'
    elif row[var] <= 750:
        return '715-750'
    elif row[var] <= 790:
        return '750-790'
    else:
        return '790+'

In [ ]:
idfc_base['naps_band'] = idfc_base.apply(naps_score_bkt,var='naps_score',axis=1)

In [ ]:
idfc_base.naps_band.value_counts(dropna=False)

In [ ]:
idfc_base[idfc_base.approved.notna()].naps_band.value_counts(dropna=False)

#### Approved case analysis

In [ ]:
def loan_amount_bkt(row, var):
    if row[var] <= 80000:
        return '<=80K'
    elif row[var] <= 100000:
        return '80K-100K'
    elif row[var] <= 200000:
        return '100K-200K'
    elif row[var] <= 300000:
        return '200K-300K'
    else:
        return '300K+'

def roi_bkt(row, var):
    if row[var] <= 19:
        return '<=19'
    elif row[var] <= 21:
        return '19-21'
    elif row[var] <= 24:
        return '21-24'
    elif row[var] <= 26:
        return '24-26'
    elif row[var] <= 28:
        return '26-28'
    else:
        return '28+'

In [ ]:
idfc_base['idfc_la_bkt'] = idfc_base.apply(loan_amount_bkt,var='loan_amount',axis=1)
idfc_base['niro_la_bkt'] = idfc_base.apply(loan_amount_bkt,var='max_amount',axis=1)

In [ ]:
idfc_base['idfc_roi_bkt'] = idfc_base.apply(roi_bkt,var='roi',axis=1)
idfc_base['niro_roi_bkt'] = idfc_base.apply(roi_bkt,var='interest_rate_perc',axis=1)

#### Disbursed

In [ ]:
idfc_base[idfc_base.loan_application_status.isin(['CLOSED','LOAN_EXECUTED'])]

#### IDFC cleanest base

In [ ]:
idfc_base['approved'] = np.where(idfc_base['approved'].isna(), "Not Approved", "Approved")

In [ ]:
clean_base = idfc_base[idfc_base['carded'].notna() & (idfc_base['MAX(EVER_DELQ)'] < 30) & (idfc_base['crif_score'] >= 700) & (idfc_base['enquiry_count_l6m'] <= 3) & (idfc_base['max_cc_util'] <= 0.65) & (idfc_base['no_tr_l6m'] <= 1)].reset_index(drop=True)

In [ ]:
idfc_base[~idfc_base.crn.isin(clean_base.crn) & idfc_base['carded'].notna()].shape

In [ ]:
clean_base.shape

In [ ]:
clean_base.approved.value_counts(dropna=False)

In [ ]:
clean_base.approved.value_counts(normalize = True, dropna=False)

In [ ]:
idfc_base[idfc_base['carded'].notna()].crif_score 

In [ ]:
idfc_base[~idfc_base.crn.isin(clean_base.crn) & idfc_base['carded'].notna()].approved.value_counts(normalize=True, dropna=False)

In [ ]:
# idfc_base[idfc_base.approved.notna()].reset_index(drop=True).to_csv('approved_base_idfc.csv')

In [ ]:
clean_base.to_excel('clean_base.xlsx')

#### Rejected base vs Approved base

In [ ]:
idfc_base.head()

In [ ]:
idfc_base[idfc_base.approved.isin(['Approved'])].shape

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Define function to remove outliers using IQR method
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

fields = ["crif_score", "no_tr_l6m", "enquiry_count_all", "enquiry_count_date_present", "enquiry_count_l6m", "enquiry_count_l3m", "MAX(EVER_DELQ)", "MAX(MAX_DELQ24)", "MAX(MAX_DELQ12)", "MAX(MAX_DELQ9)", "MAX(MAX_DELQ6)", "MAX(MAX_DELQ3)", "max_cc_util", "age"]

for field in fields:
    idfc_base_no_outliers = remove_outliers(idfc_base, field)

    # Create box plot without outliers
    plt.figure(figsize=(10, 6))
    sns.boxplot(x="approved", y=field, data=idfc_base_no_outliers)
    plt.title(field)
    plt.ylabel(field)
    plt.show()


In [ ]:
idfc_base['crif_score_bkt'] = np.where(idfc_base['crif_score'] <= 700, 'A) 650 - 700',
                                np.where(idfc_base['crif_score'] <= 720, 'B) 700 - 720',
                                    np.where(idfc_base['crif_score'] <= 750, 'C) 720 - 750',
                                        np.where(idfc_base['crif_score'] <= 800, 'D) 750 - 800',
                                            np.where(idfc_base['crif_score'] > 800, 'E) 800+',None)))))

#### CRIF vs Risk segment crosstab

In [ ]:
import pandas as pd

# Assuming 'approved' column contains 'Approved' or 'Not Approved'
# Replace 'Approved' with 1 and 'Not Approved' with 0
idfc_base['approved_binary'] = idfc_base['approved'].replace({'Approved': 1, 'Not Approved': 0})

# Create crosstab
cross_tab = pd.crosstab(index=idfc_base['crif_score_bkt'], 
                        columns=idfc_base['risk_segment'], 
                        values=idfc_base['approved_binary'], 
                        aggfunc='sum')

# Calculate total approvals for each crif_score_bkt
total_approvals = cross_tab.sum().sum()

# Calculate percentages
cross_tab_percentages = cross_tab.divide(total_approvals, axis=0) * 100

print(cross_tab_percentages)


In [ ]:
idfc_base.head()

## CIBIL data

In [ ]:
cibil_df_with_cvattrs = """
            select * from (
            select 
            distinct la.id,
            pii.gender as cibil_gender,
            pii.pin as cibil_pincode,
            pii.calculated_age,
            pii.city as cibil_city,
            pii.state as cibil_state,
            pii.native_language,
            tli.*,
            date_trunc("MONTH",date(od.created_at)) as createdate,
            rank()over(partition by od.user_id order by od.created_at asc) as rnk
            from NIRO_RDS.NIRO_DATA.CORE__PUBLIC__LOAN_APPLICATION la
            left join NIRO_RDS.NIRO_DATA.core__public__offer_user_details ou on ou.offer_id = la.offer_id
            left join ANALYTICS.OFFERS.offer_details od on od.user_id = la.user_id and od.category = ou.category
            left join ANALYTICS.OFFERS.personal_info pii on od.pii_id = pii.pii_id
            left join ANALYTICS.OFFERS.tradeline_info tli on tli.tli_id = od.tli_id
            where la.is_deleted = False
            and(la.is_deleted_at_source = false or la.is_deleted_at_source is null)
            )
where rnk = 1;
"""
cibil_df = (
    extractorFromQuery()
    .queryExecutor(cibil_df_with_cvattrs)
)
cibil_df.shape

In [ ]:
column_name_map = {'agg911': 'max_aggregate_bankcard_utilisation_l12m', 'rvlr01': 'cc_util_revolving_l1m', 'bcpmtstr': 'bank_card_payment_category', 'cv11': 'no_of_60p_accs_ever', 'cv14': 'no_of_deduped_inquiries', 
                   'mt28s': 'total_cl_open_mortgage_trades_l12m', 'mt33s': 'total_balance_open_mortgage_trades_l12m', 'pl33s':'total_bal_of_open_pl_l12m', 'at20s':'months_since_oldest_trade', 
                   'mt01s':'no_of_mortgage_trades', 'bc02s':'no_of_open_cc_trades', 'bg01s':'no_of_business_general_trades', 'cv10':'no_of_30p_accs_ever', 'trd':'no_of_trades', 'at33a':'total_bal_open_trades_l12m',
                   'au33s':'total_bal_open_auto_trades_l12m', 'co04s180':'months_since_recent_chargedoff180P', 'au28s':'total_cl_open_auto_trades_l12m', 'pl28s':'total_cl_open_pl_trades_l12m',
                   'co01s180':'no_of_chargedoff180P', 'bc28s':'total_cl_of_cc_trades_l12m', 'cv12':'no_of_90p_accs_ever', 'co05s':'total_bal_of_chargedoff_trades', 'g310s':'worst_rating_l12m',
                   'aggs911':'current_utilization_of_top_wallet_bankcard', 'at01s':'no_of_trades_pl_2w_gl_cd', 'at33a_ne_ccod':'total_bal_open_trades_l12m_except_wo_cc_od', 'cv14_12m':'no_of_deduped_inquiries_l12m',
                   'cv14_6m':'no_of_deduped_inquiries_l6m', 'cv14_3m':'no_of_deduped_inquiries_l3m', 'cv14_1m':'no_of_deduped_inquiries_l1m', 'g310s_24m':'worst_rating_l24m',
                   'g310s_6m':'worst_rating_l6m', 'g310s_3m':'worst_rating_l3m', 'g310s_1m':'worst_rating_l1m', 'cv11_24m':'no_of_60p_accs_l24m', 'cv11_12m':'no_of_60p_accs_l12m', 'g057s_1dpd_36m': 'no_of_trades_bounced_l36m',
                   'g057s_1dpd_12m':'no_of_trades_bounced_l12m', 'bc106s_60dpd':'no_of_cc_trades_60P_l24m', 'bc107s_24m':'no_of_30P_cc_l24m', 'bc106s_60dpd_12m':'no_of_cc_trades_60P_l12m', 'bc107s_12m':'no_of_30P_cc_l12m',
                   'bc106s_le_30dpd_12m':'no_of_cc_lt30p_l12m', 'bc09s_36m_hcsa_le_30':'no_cc_sanctionedamt_lt30K_l36m', 'pl09s_36m_hcsa_le_30':'no_pl_sanctionedamt_lt30K_l36m', 'at09s_6m':'no_tr_opened_l6m',
                   'g310s_36m':'worst_dpd_l36m', 'at33a_ne_wo':'total_bal_open_trades_l12m_minus_total_bal_of_chargedoff_trades', 'at09s_12m':'no_tr_opened_l12m', 'at09s_3m':'no_tr_opened_l3m', 'cv13':'per_of_30p_accs_ever',
                    'cv24':'total_payment_amount_bankcard_accs_l3m', 'revs904':'max_revolving_monthly_spend_l12m', 'cv20':'total_monthly_obligations_l3m', 'cv22':'total_balance_bankcard_accs_l3m', 
                    'ul_trd':'no_of_trades_unsec', 'cv21':'total_payment_amount_l3m', 'g310s_2m':'worst_dpd_l2m'}

In [ ]:
cibil_df.rename(columns = column_name_map, inplace=True)
cibil_df.drop_duplicates(subset=["id"], inplace=True)

In [ ]:
idfc_base = idfc_base.merge(cibil_df, left_on='crn', right_on='hashed_phone', how = 'left')